In [1]:
# This row works only with SparkMagic
%%info

SyntaxError: invalid syntax (<ipython-input-1-9cf2458b3e8e>, line 2)

In [3]:
import os
import pyspark
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk-pom:1.10.34,org.apache.hadoop:hadoop-aws:2.7.2 pyspark-shell'
from pyspark.sql import SQLContext
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [5]:
filePath = "s3a://mle7-data/train_numeric.csv"
df = sqlContext.read.load(filePath,format="csv", inferSchema="true", header="true").cache()

In [4]:
### Data exploration
df.printSchema()

#df.count()

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


#firstrows=df.take(5) # this one returns a list, not a dataframe!
firstrowsdf=df.limit(50)
firstrowsdf

# Print with pandas
import pandas as pd

#pd.DataFrame(firstrows, columns=df.columns).transpose()
firstrowsdf.toPandas().transpose()

In [5]:
#df.describe().toPandas().transpose()
#firstrowsdf.describe()

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [6]:
featureColumnNames=df.schema.names[1:969]
featureColumnNames.transpose()

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [7]:
# take a subset of data
mldf=df.limit(50)
#mldf.describe().show(3)

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [8]:
#mldf.na.drop().take(3)

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [9]:
#features=mldf.select([c for c in mldf.columns if c in featureColumnNames])
#labels=mldf.select(["Response"])

from pyspark.ml.feature import FeatureHasher

#hasher = FeatureHasher(inputCols=['L0_S0_F14', 'L0_S0_F18'],
#                       outputCol="features")

hasher = FeatureHasher(inputCols=featureColumnNames,
                       outputCol="features")

featurized = hasher.transform(mldf)
featurized.show(truncate=False)

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [10]:
#from pyspark.ml.feature import VectorAssembler
#vectorAssembler = VectorAssembler(inputCols = featureColumnNames, outputCol = 'features')
#vectorAssembler = VectorAssembler(inputCols = ['L0_S0_F14', 'L0_S0_F18'], outputCol = 'features')
#prepped_df = vectorAssembler.transform(mldf)
#prepped_df = prepped_df.select(['features', 'Response'])

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [11]:
prepped_df=featurized.select(['features','Response'])

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [12]:
prepped_df.take(3)

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [13]:
splits = prepped_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [14]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='Response', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.
